In [1]:
from nltk.parse.corenlp import CoreNLPParser

parser = CoreNLPParser(url="http://localhost:9000")

line = "the furry lover"

CorenlpGrammarTree = next(parser.raw_parse(line))

import nltk

def tree_to_cfg(tree):
    cfg = {}

    def traverse(node):
        if isinstance(node, str):  # Terminal node
            return [node]
        else:
            children = [traverse(child) for child in node]
            non_terminal = node.label()
            if non_terminal not in cfg:
                cfg[non_terminal] = []
            cfg[non_terminal].extend(children)
            return [non_terminal]

    traverse(tree)
    return cfg

# Parse tree in the string format
parse_tree_str = str(CorenlpGrammarTree)

# Convert string to nltk.Tree object
parse_tree = nltk.Tree.fromstring(parse_tree_str)

# Convert the parse tree to CFG
cfg = tree_to_cfg(parse_tree)

# Print the CFG rules
for non_terminal, expansions in cfg.items():
    print(non_terminal + " -> " + " | ".join([" ".join(expansion) for expansion in expansions]))

DT -> the
JJ -> furry
NN -> lover
NP -> DT | JJ | NN
ROOT -> NP
